# ECH Workers RS - Colab Build

This notebook builds the ECH Workers RS project on Google Colab and pushes changes to GitHub.

## Prerequisites
- GitHub SSH key (will be configured below)
- Repository access to `justinwoo280/ech-workers-rs`

## 1. Configure SSH Key for GitHub

**Important**: Paste your GitHub SSH private key below (the content of `~/.ssh/id_ed25519` or `~/.ssh/id_rsa`)

In [ ]:
#@title SSH Key Configuration { display-mode: "form" }
#@markdown Paste your SSH private key here:
ssh_private_key = ""  #@param {type:"string"}

import os

# Create .ssh directory
os.makedirs(os.path.expanduser('~/.ssh'), exist_ok=True)

# Write SSH key
if ssh_private_key:
    with open(os.path.expanduser('~/.ssh/id_ed25519'), 'w') as f:
        f.write(ssh_private_key)
    os.chmod(os.path.expanduser('~/.ssh/id_ed25519'), 0o600)
    print("✅ SSH key configured")
else:
    print("⚠️ No SSH key provided. You can also use the next cell to upload key file.")

In [ ]:
#@title Alternative: Upload SSH Key File
from google.colab import files
import os

print("Upload your SSH private key file (id_ed25519 or id_rsa):")
uploaded = files.upload()

for filename, content in uploaded.items():
    key_path = os.path.expanduser(f'~/.ssh/{filename}')
    with open(key_path, 'wb') as f:
        f.write(content)
    os.chmod(key_path, 0o600)
    print(f"✅ SSH key saved to {key_path}")

In [ ]:
#@title Configure SSH for GitHub
%%bash
# Add GitHub to known hosts
ssh-keyscan -t ed25519 github.com >> ~/.ssh/known_hosts 2>/dev/null
ssh-keyscan -t rsa github.com >> ~/.ssh/known_hosts 2>/dev/null

# Create SSH config
cat > ~/.ssh/config << 'EOF'
Host github.com
    HostName github.com
    User git
    IdentityFile ~/.ssh/id_ed25519
    IdentitiesOnly yes
EOF

chmod 600 ~/.ssh/config
echo "✅ SSH config created"

## 2. Install Build Tools

In [ ]:
#@title Install Zig 0.13.0
%%bash
set -e

ZIG_VERSION="0.13.0"
ZIG_DIR="/opt/zig"

if [ ! -f "$ZIG_DIR/zig" ]; then
    echo "📦 Downloading Zig $ZIG_VERSION..."
    wget -q "https://ziglang.org/download/$ZIG_VERSION/zig-linux-x86_64-$ZIG_VERSION.tar.xz" -O /tmp/zig.tar.xz
    
    echo "📂 Extracting..."
    sudo mkdir -p $ZIG_DIR
    sudo tar -xf /tmp/zig.tar.xz -C $ZIG_DIR --strip-components=1
    rm /tmp/zig.tar.xz
fi

# Add to PATH
echo "export PATH=$ZIG_DIR:\$PATH" >> ~/.bashrc
export PATH=$ZIG_DIR:$PATH

$ZIG_DIR/zig version
echo "✅ Zig installed"

In [ ]:
#@title Install Rust
%%bash
set -e

if ! command -v rustc &> /dev/null; then
    echo "📦 Installing Rust..."
    curl --proto '=https' --tlsv1.2 -sSf https://sh.rustup.rs | sh -s -- -y --default-toolchain stable
fi

source $HOME/.cargo/env
rustc --version
cargo --version
echo "✅ Rust installed"

In [ ]:
#@title Install Build Dependencies
%%bash
set -e

echo "📦 Installing build dependencies..."
sudo apt-get update -qq
sudo apt-get install -y -qq cmake ninja-build clang libclang-dev pkg-config libssl-dev

echo "✅ Dependencies installed"

## 3. Clone Repository

In [ ]:
#@title Clone Repository
%%bash
set -e

cd /content

if [ -d "ech-workers-rs" ]; then
    echo "📂 Repository already exists, pulling latest..."
    cd ech-workers-rs
    git pull
    git submodule update --init --recursive
else
    echo "📥 Cloning repository..."
    git clone --recursive git@github.com:justinwoo280/ech-workers-rs.git
    cd ech-workers-rs
fi

# Configure git
git config user.email "build@colab.google.com"
git config user.name "Colab Build"

echo "✅ Repository cloned"
ls -la

## 4. Build BoringSSL

In [ ]:
#@title Build BoringSSL
%%bash
set -e

cd /content/ech-workers-rs/zig-tls-tunnel/vendor/boringssl

if [ -f "build/libssl.a" ] && [ -f "build/libcrypto.a" ]; then
    echo "✅ BoringSSL already built"
    ls -lh build/lib*.a
else
    echo "🔨 Building BoringSSL..."
    mkdir -p build
    cd build
    
    cmake -GNinja \
        -DCMAKE_BUILD_TYPE=Release \
        -DCMAKE_C_COMPILER=clang \
        -DCMAKE_CXX_COMPILER=clang++ \
        ..
    
    ninja -j$(nproc)
    
    echo "✅ BoringSSL built"
    ls -lh lib*.a
fi

## 5. Build Zig TLS Tunnel

In [ ]:
#@title Build Zig TLS Tunnel
%%bash
set -e

export PATH=/opt/zig:$PATH

cd /content/ech-workers-rs/zig-tls-tunnel

echo "🔨 Building Zig TLS Tunnel..."
zig build -Doptimize=ReleaseFast

echo "✅ Zig TLS Tunnel built"
ls -lh zig-out/lib/

## 6. Build Rust Client

In [ ]:
#@title Build Rust Client
%%bash
set -e

source $HOME/.cargo/env
export PATH=/opt/zig:$PATH

cd /content/ech-workers-rs/ech-workers-rs

echo "🔨 Building Rust client..."
cargo build --release 2>&1 | tail -20

echo ""
echo "✅ Rust client built"
ls -lh target/release/ech-workers-rs 2>/dev/null || echo "Binary not found (check build output)"

In [ ]:
#@title Strip Binary (Optional - Reduces Size)
%%bash
set -e

cd /content/ech-workers-rs/ech-workers-rs

BINARY="target/release/ech-workers-rs"

if [ -f "$BINARY" ]; then
    echo "Before strip:"
    ls -lh $BINARY
    
    strip $BINARY
    
    echo ""
    echo "After strip:"
    ls -lh $BINARY
else
    echo "Binary not found"
fi

## 7. Git Operations

In [ ]:
#@title Check Git Status
%%bash
cd /content/ech-workers-rs

echo "📊 Git Status:"
git status

echo ""
echo "📝 Changed files:"
git diff --name-only

In [ ]:
#@title Commit and Push Changes { display-mode: "form" }
#@markdown Enter commit message:
commit_message = "chore: optimize Zig build and disable auto_ech"  #@param {type:"string"}

import subprocess
import os

os.chdir('/content/ech-workers-rs')

# Add all changes
print("📦 Adding changes...")
subprocess.run(['git', 'add', '-A'], check=True)

# Check if there are changes to commit
status = subprocess.run(['git', 'status', '--porcelain'], capture_output=True, text=True)
if not status.stdout.strip():
    print("ℹ️ No changes to commit")
else:
    # Commit
    print(f"📝 Committing: {commit_message}")
    subprocess.run(['git', 'commit', '-m', commit_message], check=True)
    
    # Push
    print("🚀 Pushing to GitHub...")
    result = subprocess.run(['git', 'push', 'origin', 'main'], capture_output=True, text=True)
    
    if result.returncode == 0:
        print("✅ Successfully pushed to GitHub!")
    else:
        print(f"❌ Push failed: {result.stderr}")
        # Try with master branch
        print("Trying 'master' branch...")
        result = subprocess.run(['git', 'push', 'origin', 'master'], capture_output=True, text=True)
        if result.returncode == 0:
            print("✅ Successfully pushed to GitHub (master branch)!")
        else:
            print(f"❌ Push failed: {result.stderr}")

## 8. Download Artifacts (Optional)

In [ ]:
#@title Download Built Binary
from google.colab import files
import os

binary_path = '/content/ech-workers-rs/ech-workers-rs/target/release/ech-workers-rs'

if os.path.exists(binary_path):
    print(f"📦 Downloading binary ({os.path.getsize(binary_path) / 1024 / 1024:.1f} MB)...")
    files.download(binary_path)
else:
    print("❌ Binary not found. Build may have failed.")

In [ ]:
#@title Download Zig Library
from google.colab import files
import os

lib_path = '/content/ech-workers-rs/zig-tls-tunnel/zig-out/lib/libzig-tls-tunnel.a'

if os.path.exists(lib_path):
    print(f"📦 Downloading Zig library ({os.path.getsize(lib_path) / 1024:.1f} KB)...")
    files.download(lib_path)
else:
    print("❌ Library not found.")

---
## Quick Build (All-in-One)

Run all build steps in sequence:

In [ ]:
#@title 🚀 Quick Build (All Steps)
%%bash
set -e

echo "========================================"
echo "ECH Workers RS - Full Build"
echo "========================================"

# Install Zig
ZIG_DIR="/opt/zig"
if [ ! -f "$ZIG_DIR/zig" ]; then
    echo "📦 Installing Zig..."
    wget -q "https://ziglang.org/download/0.13.0/zig-linux-x86_64-0.13.0.tar.xz" -O /tmp/zig.tar.xz
    sudo mkdir -p $ZIG_DIR
    sudo tar -xf /tmp/zig.tar.xz -C $ZIG_DIR --strip-components=1
    rm /tmp/zig.tar.xz
fi
export PATH=$ZIG_DIR:$PATH

# Install Rust
if ! command -v rustc &> /dev/null; then
    echo "📦 Installing Rust..."
    curl --proto '=https' --tlsv1.2 -sSf https://sh.rustup.rs | sh -s -- -y --default-toolchain stable
fi
source $HOME/.cargo/env

# Install dependencies
echo "📦 Installing dependencies..."
sudo apt-get update -qq
sudo apt-get install -y -qq cmake ninja-build clang libclang-dev pkg-config libssl-dev

cd /content

# Clone if needed
if [ ! -d "ech-workers-rs" ]; then
    echo "📥 Cloning repository..."
    git clone --recursive git@github.com:justinwoo280/ech-workers-rs.git
fi
cd ech-workers-rs

# Build BoringSSL
if [ ! -f "zig-tls-tunnel/vendor/boringssl/build/libssl.a" ]; then
    echo "🔨 Building BoringSSL..."
    cd zig-tls-tunnel/vendor/boringssl
    mkdir -p build && cd build
    cmake -GNinja -DCMAKE_BUILD_TYPE=Release -DCMAKE_C_COMPILER=clang -DCMAKE_CXX_COMPILER=clang++ ..
    ninja -j$(nproc)
    cd /content/ech-workers-rs
fi

# Build Zig
echo "🔨 Building Zig TLS Tunnel..."
cd zig-tls-tunnel
zig build -Doptimize=ReleaseFast
cd ..

# Build Rust
echo "🔨 Building Rust client..."
cd ech-workers-rs
cargo build --release 2>&1 | tail -5

# Strip
if [ -f "target/release/ech-workers-rs" ]; then
    strip target/release/ech-workers-rs
    echo ""
    echo "========================================"
    echo "✅ Build Complete!"
    echo "========================================"
    ls -lh target/release/ech-workers-rs
else
    echo "❌ Build failed"
fi